# Check GPU version.

In [ ]:
!nvidia-smi

# Mount google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install TensorFlow-1.14 GPU.

In [ ]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y 1>/dev/null 2>/dev/null 
!pip uninstall tensorflow-gpu -y 1>/dev/null 2>/dev/null 

# Install TensorFlow-1.14.
!pip install --upgrade tensorflow==1.14.0 1>/dev/null 2>/dev/null 
!pip install --upgrade tensorflow-gpu==1.14.0 1>/dev/null 2>/dev/null 

TensorFlow 1.x selected.


# Restart the runtime.

# Set the root directory.

In [89]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

total 24
drwxr-xr-x 1 root root 4096 Jun 30 10:01 .
drwxr-xr-x 1 root root 4096 Jun 30 08:23 ..
drwxr-xr-x 1 root root 4096 Jun 26 16:26 .config
drwxr-xr-x 2 root root 4096 Jun 30 09:26 .ipynb_checkpoints
drwxr-xr-x 1 root root 4096 Jun 26 16:26 sample_data
drwxr-xr-x 7 root root 4096 Jun 30 10:01 VMWorkloadPredictor


# Import TensorFlow-1.14.

In [90]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import backend as K

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

1.15.2


# Clone VM workload dataset repository.

In [91]:
!rm -rf VMWorkloadPredictor

In [92]:
!git clone https://github.com/look4pritam/VMWorkloadPredictor.git

Cloning into 'VMWorkloadPredictor'...
remote: Enumerating objects: 2535, done.
remote: Total 2535 (delta 0), reused 0 (delta 0), pack-reused 2535
Receiving objects: 100% (2535/2535), 25.17 MiB | 6.86 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Checking out files: 100% (2524/2524), done.


In [93]:
!ls -al

total 24
drwxr-xr-x 1 root root 4096 Jun 30 10:06 .
drwxr-xr-x 1 root root 4096 Jun 30 08:23 ..
drwxr-xr-x 1 root root 4096 Jun 26 16:26 .config
drwxr-xr-x 2 root root 4096 Jun 30 09:26 .ipynb_checkpoints
drwxr-xr-x 1 root root 4096 Jun 26 16:26 sample_data
drwxr-xr-x 7 root root 4096 Jun 30 10:06 VMWorkloadPredictor


In [ ]:
!ls -al VMWorkloadPredictor/output
!ls -al VMWorkloadPredictor/ml_data

# Set dataset root directory.

In [95]:
dataset_path = '/content/VMWorkloadPredictor/ml_data/*.csv'

# Load dataset.

In [96]:
import glob
import csv

In [99]:
sample_history = 10

def load_dataset(dataset_path):
    data = []
    labels = []

    print('Load dataset - Start.')

    for fname in glob.glob(dataset_path):
        with open(fname, 'r') as infh:
            reader = csv.reader(infh, delimiter=';')

            for row in reader:
                r = np.array(row, dtype = float)
                rr = []
                for i in range(sample_history):
                    rr.append(r[i * 7 + 1])
                    rr.append(r[i * 7 + 2])
                #print(rr)
                data.append(rr)
                labels.append(r[-1])
 
    data = np.array(data)
    labels = np.array(labels)
    n = int(float(data.shape[0]) * 0.8)
    train_data = data[:n]
    train_labels = labels[:n]
    test_data = data[n:]
    test_labels = labels[n:]
    print('Load dataset - End.')
    return train_data, train_labels, test_data, test_labels

In [100]:
train_data, train_labels, test_data, test_labels = load_dataset(dataset_path)

Load dataset - Start.
Load dataset - End.


In [34]:
!python preprocess.py

287
287
287
287
287
579
287
286
287
287
288
287
287
287
287
287
287
287
287
538
287
287
287
287
287
287
287
287
287
287
287
58
287
287
287
287
287
64
675
287
376
287
287
287
287
287
141
287
287
287
287
287
287
288
287
287
579
287
287
376
287
287
287
287
287
287
287
287
287
287
287
287
287
287
287
276
287
287
287
220
287
0
287
287
287
287
287
287
288
287
287
287
287
287
537
287
287
287
287
287
249
287
287
287
287
287
58
287
218
287
287
287
287
287
287
287
287
287
287
287
287
287
287
202
287
288
287
287
287
789
287
287
287
287
287
287
287
287
376
287
287
287
249
287
287
287
287
288
287
287
287
287
287
287
287
345
287
249
287
287
287
287
287
579
287
376
287
287
288
287
287
311
211
287
578
287
287
287
287
287
287
287
287
287
287
287
287
69
288
287
287
287
287
287
287
287
287
675
287
278
287
287
287
287
287
287
287
287
287
287
287
287
287
287
376
287
287
579
287
287
287
287
287
278
287
287
287
287
287
287
287
287
287
287
287
287
249
287
675
287
287
578
126
268
287
287
288
287
287
57
287
287

In [4]:
!python main_cpu.py

loading data...
finished loading data
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
Best: MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, vali